<a href="https://colab.research.google.com/github/carlos-alves-one/-Amazon-Review-NLP/blob/main/Sentiment_Analysis_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Goldsmiths University of London
### MSc. Data Science and Artificial Intelligence
### Module: Natural Language Processing
### Author: Carlos Manuel De Oliveira Alves
### Student: cdeol003
### Coursework Project

# Data Collection

### Load the data

In [1]:
# Imports the 'drive' module from 'google.colab' and mounts the Google Drive to
# the '/content/drive' directory in the Colab environment.
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Dataset source: https://www.kaggle.com/datasets/akudnaver/amazon-reviews-dataset

License: Unknown

In [23]:
# Import the pandas library and give it the alias 'pd' for data manipulation and analysis
import pandas as pd

# Load the dataset Amazon Review Details from Google Drive
data_path = '/content/drive/MyDrive/amazon_project/amazon-review-details.csv'
data = pd.read_csv(data_path)

# Display the first few rows of the dataframe
data.head(3).T


,0,1,2
report_date,2019-01-02,2019-01-03,2019-01-03
online_store,FRESHAMAZON,FRESHAMAZON,FRESHAMAZON
upc,8718114216478,5000184201199,5000184201199
retailer_product_code,B0142CI6FC,B014DFNNRY,B014DFNNRY
brand,Dove Men+Care,Marmite,Marmite
category,Personal Care,Foods,Foods
sub_category,Deos,Savoury,Savoury
product_description,Dove Men+Care Extra Fresh Anti-perspirant Deod...,Marmite Spread Yeast Extract 500g,Marmite Spread Yeast Extract 500g
review_date,2019-01-01,2019-01-02,2019-01-02
review_rating,5,5,4


# Data Preprocessing

The dataset contains multiple columns, but for our sentiment analysis, we will primarily focus on the 'review_rating' as our target variable and the text of the reviews for our feature.

**Tasks :**

- Select relevant columns ('review_rating' and the review text column).

- Handle missing values if necessary.

- Convert ratings to a binary sentiment (positive or negative).

- Preprocess the text data (tokenization, lowercasing, removing stop words, etc.).


## Import Libraries and Packages

In [29]:
# Importing the 'stopwords' collection from the nltk.corpus module
from nltk.corpus import stopwords

# Importing the 're' module for regular expression operations
import re

# Importing the 'word_tokenize' function from nltk.tokenize for tokenizing strings into words
from nltk.tokenize import word_tokenize

# Importing the nltk module, which is a suite of libraries for natural language processing
import nltk

# Downloading the 'punkt' tokenizer models, used by nltk for sentence tokenization
nltk.download('punkt')

# Downloading the 'stopwords' dataset, which contains lists of common stopwords in various languages
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Function for Cleaning & Preprocessing

In [30]:
# Declare function for data cleaning and preprocessing
def preprocess_text(text):

    # Lowercasing
    text = text.lower()

    # Remove punctuation and numbers
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Returns a string where all elements in the list 'tokens'
    # are concatenated into a single string, separated by spaces
    return ' '.join(tokens)


## Preprocessing the Review Text

In [33]:
# Apply preprocessing to the review text
data['processed_reviews'] = data['review_text'].apply(preprocess_text)


## Create Column with Binary Sentiment

In [31]:
# Convert ratings to binary sentiment
data['sentiment'] = data['review_rating'].apply(lambda x: 1 if x > 3 else 0)


In [32]:
data.head(3).T

,0,1,2
report_date,2019-01-02,2019-01-03,2019-01-03
online_store,FRESHAMAZON,FRESHAMAZON,FRESHAMAZON
upc,8718114216478,5000184201199,5000184201199
retailer_product_code,B0142CI6FC,B014DFNNRY,B014DFNNRY
brand,Dove Men+Care,Marmite,Marmite
category,Personal Care,Foods,Foods
sub_category,Deos,Savoury,Savoury
product_description,Dove Men+Care Extra Fresh Anti-perspirant Deod...,Marmite Spread Yeast Extract 500g,Marmite Spread Yeast Extract 500g
review_date,2019-01-01,2019-01-02,2019-01-02
review_rating,5,5,4
